In [1]:
# find simulation files
import glob
filepath_list = glob.glob("exercise-1/d3plots/*.d3plot")
len(filepath_list)

100

In [2]:
# data extraction
from lasso.dyna import D3plot
# pip install lasso-python

# just for info
d3plot = D3plot(filepath_list[0])
d3plot.arrays["node_displacement"].shape
print("timesteps, nodes, xyz")
print(d3plot.arrays["node_displacement"].shape)

fields = []
last_timestep = -1

for filepath in filepath_list:
    # open file
    d3plot = D3plot(filepath)
    
    # extract result field
    node_displacement = d3plot.arrays["node_displacement"][last_timestep]
    
    # collect in list
    fields.append(node_displacement)
    
print("n_simulations", len(fields))

timesteps, nodes, xyz
(5, 2500, 3)
n_simulations 100


In [3]:
import numpy as np

# stack arrays and bring them into form
data = np.stack(fields)
data = data.reshape((100, 2500*3))
data.shape

(100, 7500)

In [13]:
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, MDS
# pip install sklearn

# reduce dimensions to 3 for plotting
#model = PCA(n_components=3)
#model.fit(data)

#model = LocallyLinearEmbedding(n_components=3)
#model.fit(data)

model = MDS(n_components=3)
model.fit(data)

MDS(n_components=3)

In [11]:
import plotly.graph_objects as go
fig = go.FigureWidget()
fig.add_trace(
    go.Scatter(x=np.arange(len(model.singular_values_)),
               y=model.singular_values_,
               mode='lines+markers',
               name="PCA Singular Values")
)

fig.show()

AttributeError: 'LocallyLinearEmbedding' object has no attribute 'singular_values_'

In [19]:
#data_transformed = model.transform(data)
#data_transformed.shape

data_x_transformed = model.embedding_

In [16]:
import ntpath

# collect image data
image_data = {}
# {
#   14: b'38904gjg3j50g9...',
#   ...
# }

# exercise-1/images/14.png
# basename: 14.png
# split: ["14", "png"]

for img_filepath in glob.glob("exercise-1/images/*.png"):
    with open(img_filepath, "rb") as fp:
        number_txt = ntpath.basename(img_filepath).split(".")[0]
        run_id = int(number_txt)
        image_data[run_id] = fp.read()

In [20]:
from ipywidgets import Image, Layout, HBox

image_widget = Image(
    value=image_data[1],
    layout=Layout(height="500px", width="375px")
)

fig = go.FigureWidget(
    data=[
        dict(
            type="scatter3d",
            x=data_transformed[:, 0],
            y=data_transformed[:, 1],
            z=data_transformed[:, 2],
            mode="markers",
            marker= {
                "size": 5
            },
            hoverinfo="text",
            text=filepath_list,
        )
    ]
)

def hover_fn(trace, points, state):
    
    ind = points.point_inds[0]
    filepath = filepath_list[ind]
    # run14.d3plot
    # ...
    # as before: run14
    # 14
    number_txt = ntpath.basename(filepath).split(".")[0].replace("run","")
    run_id = int(number_txt)
    image_widget.value = image_data[run_id]

# register hove function
scatter = fig.data[0]
scatter.on_hover(hover_fn)

fig.update_layout(
    margin=go.layout.Margin(l=0, r=0, b=0, t=0),
    hovermode="closest",
)

HBox([fig, image_widget])

    'data': [{'hoverinfo': 'text',
              'marker': {'size': 5},
        …